# Notebook for extracting tabular datasets to estimate AGBD for Kenya
1. Planet + GEDI (biomass) for Kenya
2. Sentinel + GEDI (biomass) for Kenya
3. Download GEDI Image data for Netherlands/Kenya
4. Additional covariates for Kenya
5. Extract image data used for biomass inference - 61 variables
6. Extract image data for exploring enviromental conditions for field sites
7. Extract image data for exploring enviromental conditions for SOC field sites

#### Import Libraries

In [2]:
%load_ext watermark

In [1]:
import os

import ee, eemont, geemap
from geeml.utils import eeprint, getCountry, createGrid
from geeml.extract import extractor
import geedim
from tqdm.auto import tqdm

import pandas as pd
import fnmatch
import warnings
warnings.filterwarnings("ignore")

#### Authenticate (once per week if local) and alqays if on colab

In [4]:
# ee.Authenticate()
ee.Initialize()

#### Print python and package versions

In [5]:
# Print 1) Venv version and 2)imported package versions 
%watermark -v -m --iversions

Python implementation: CPython
Python version       : 3.9.12
IPython version      : 8.5.0

Compiler    : MSC v.1916 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : Intel64 Family 6 Model 165 Stepping 2, GenuineIntel
CPU cores   : 12
Architecture: 64bit

ee    : 0.1.327
geedim: 1.5.3
pandas: 1.5.0
eemont: 0.3.5
geemap: 0.17.1



#### Define the area of interest (kenya) and a vegetation mask (removes buildings and open water)

In [6]:
# Create a point within Kenya
aoi = ee.Geometry.Point([37.857884,-0.002197])
# Load simplified country boundaries from GEE
countries = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')
# use defined point to select Kenya boundary
kenya = countries.filterBounds(aoi)

# Create a mask to remove building and water pixels as based on the worldcover dataset
landcover = ee.ImageCollection("ESA/WorldCover/v100")
lcmask = landcover.filterBounds(kenya).mosaic().neq([50, 70, 80]).reduce(ee.Reducer.max())# buildings, snow/ice/ and open water
# eeprint(lcmask)

In [7]:
# Load GEDI biomass data
l4b = ee.Image('LARSE/GEDI/GEDI04_B_002')
# Select mean biomass band/update to extent of vegetation and kenya boundary. Rename to 'biomass'.
target = l4b.select('MU').updateMask(lcmask).rename('biomass').clip(kenya)

## 1. Extract Planet data at GEDI Biomass points within Kenya

In [ ]:
# Planet data
planet = ee.ImageCollection("projects/planet-nicfi/assets/basemaps/africa")
# Compute percentile RGB NIR bands for planet data
monthlyPlanet = planet.filterBounds(kenya).filter(ee.Filter.eq('cadence','monthly'))\
                        .reduce(ee.Reducer.percentile([5,25,50,75,95]))

In [ ]:
# Extract Planet data
dd = r'C:\Users\coach\myfiles\conservation\Handover\AGBD\Data\inputs'
biomassextractor = extractor(monthlyPlanet.addBands(target), kenya.geometry(), 1000, dd, target, 50000, 25)

# For GEDI pixels, extract planet data at pixels
biomassextractor.extractPoints(gridSize = 20000, batchSize = 10000, filename = 'biomassPlanet.csv')

## 2. Extract Sentinel-2 data at GEDI Biomass points within Kenya

In [17]:
# Sentinel
s2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
# Compute percentile RGB NIR bands for sentinel data
covariates = s2.filterBounds(kenya).filterDate('2019-04-18', '2021-08-04').select(['B4', 'B3', 'B2', 'B8'])\
    .reduce(ee.Reducer.percentile([5,25,50,75,95]))

In [18]:
#  Extract Sentinel data
dd = r'C:\Users\coach\myfiles\conservation\Handover\AGBD\Data\inputs'
biomassextractor = extractor(covariates.addBands(target), kenya.geometry(), 1000, dd, target, 50000, 25)

# For GEDI pixels, extract sentinel data at pixels
biomassextractor.extractPoints(gridSize = 30000, batchSize = 15000, filename = 'biomassS2.csv')

Points: |                                                                              | [  0.0%] in 00:00 (et…

## 3. (Optional) Download Image (biomass) GEDI data (1km spatial resolution) for Kenya

In [6]:
# poi = ee.Geometry.Point([5.627073,51.900901]) # Netherlands
poi = ee.Geometry.Point([37.857884,-0.002197])# Kenya
aoi = getCountry(poi)

In [7]:
landcover = ee.ImageCollection("ESA/WorldCover/v100")
lcmask = landcover.filterBounds(aoi).mosaic().neq([50, 70, 80]).reduce(ee.Reducer.max())# buildings, snow/ice/ and open water
# eeprint(lcmask)
l4b = ee.Image('LARSE/GEDI/GEDI04_B_002')
target = l4b.select('MU').updateMask(lcmask).rename('biomass').clip(aoi)

filename = Path(r'C:\Users\coach\myfiles\conservation\Handover\TCH\Data\inputs\DNN\Y_GEDI_biomass.tif')
BaseImage(target.unmask(0).clip(aoi)).download(filename, crs='EPSG:4326', region= aoi.geometry(), scale=1000, overwrite=True, num_threads=20)

Y_GEDI_biomass.tif: |                                                 | 0.00/3.86M (raw) [  0.0%] in 00:00 (et…

## 4. Additional covaraiates to extract from GEE

#### Define spatio-temporal parameters for computing covariates

In [8]:
# period of GEDI biomass data
startDate = '2019-04-18'
endDate = '2021-08-04'

# Kenya extent
poi = ee.Geometry.Point([37.857884,-0.002197])
countries = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')
kenya = countries.filterBounds(poi)

### Optical

In [9]:
# PlanetScope data - VNIR percetile data (Not currently included in final covariates list)
planet = ee.ImageCollection("projects/planet-nicfi/assets/basemaps/africa")
monthlyPlanet = planet.filterBounds(kenya).filter(ee.Filter.eq('cadence','monthly'))\
                    .filterDate(startDate, endDate)\
                    .reduce(ee.Reducer.percentile([5,25,50,75,95]))

In [10]:
# Sentinel-2 data filtered for scenes with less tha 35% cover and scaling factors applied.
s2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
s2_filtered = s2.filterDate(startDate, endDate).filterBounds(kenya).filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', 35))\
.map(lambda i: i.divide(10000))

In [11]:
# Landsat- 8 and Landsat-9

# Apply scaling factors.
def landsatScale(image):
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    return image.addBands(opticalBands, None, True)\
              .addBands(thermalBands, None, True)

l8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
l8_filtered = l8.filterDate(startDate, endDate).filterBounds(kenya)\
.filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', 35)).map(landsatScale)

l9 = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2')
l9_filtered = l9.filterDate(startDate, endDate).filterBounds(kenya)\
.filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', 35)).map(landsatScale)

In [12]:
# Compute median s2, l8 and l9 images
S2 = s2.select('B.*').median().regexpRename('$', '_s2')
L8 = l8.select(["SR_B1","SR_B2","SR_B3","SR_B4","SR_B5","SR_B6","SR_B7","ST_B10"])\
.median().regexpRename('$', '_l8')
L9 = l9.select(["SR_B1","SR_B2","SR_B3","SR_B4","SR_B5","SR_B6","SR_B7","ST_B10"])\
.median().regexpRename('$', '_l9')

In [13]:
# Compute NDVI - uses eemont
S2VI = s2_filtered.spectralIndices('NDVI', drop=True).median().regexpRename('$', '_s2')

In [10]:
eeprint(S2VI.bandNames())

### Radar

In [14]:
# Sentinel-1
# Global normalised backscatter
VH = ee.ImageCollection("projects/sat-io/open-datasets/S1GBM/normalized_s1_backscatter_VH")\
.filterBounds(kenya).mosaic().clip(kenya).rename('normalized_VH')
VV = ee.ImageCollection("projects/sat-io/open-datasets/S1GBM/normalized_s1_backscatter_VV")\
.filterBounds(kenya).mosaic().clip(kenya).rename('normalized_VV')

In [15]:
# Alos palsar-2
def scaleAlos(image):
    return image.pow(2).log10().multiply(10).subtract(83)

Palsar = ee.ImageCollection("JAXA/ALOS/PALSAR/YEARLY/SAR_EPOCH").filterBounds(aoi)\
.select(['HH', 'HV']).map(scaleAlos).mosaic()

### Tree related variables

In [16]:
# tree canopy layer (brandt)
tree_canopy = ee.ImageCollection('projects/wri-datalab/TML').filterBounds(kenya).mosaic().rename('tree_canopy')

# tree canopy height (lang)
canopy_height = ee.Image("users/nlang/ETH_GlobalCanopyHeight_2020_10m_v1").clip(kenya).rename('lang_CHM')
ch_standard_deviation = ee.Image("users/nlang/ETH_GlobalCanopyHeightSD_2020_10m_v1").clip(kenya).rename('lang_CHM_sd')

# tree type (decidous, broad leaf, needleaf)
tree_type = ee.ImageCollection("MODIS/006/MCD12Q1").filterBounds(kenya).select('LC_Type1').mosaic()

# Global consensus landcover
barren = ee.Image("projects/sat-io/open-datasets/global_consensus_landcover/barren")
cultivated_and_managed_vegetation = ee.Image("projects/sat-io/open-datasets/global_consensus_landcover/cultivated_and_managed_vegetation");
deciduous_broadleaf_trees = ee.Image("projects/sat-io/open-datasets/global_consensus_landcover/deciduous_broadleaf_trees");
evergreen_deciduous_needleleaf_trees = ee.Image("projects/sat-io/open-datasets/global_consensus_landcover/evergreen-deciduous_needleleaf_trees");
evergreen_broadleaf_trees = ee.Image("projects/sat-io/open-datasets/global_consensus_landcover/evergreen_broadleaf_trees");
herbaceous_vegetation = ee.Image("projects/sat-io/open-datasets/global_consensus_landcover/herbaceous_vegetation");
mixed_other_trees = ee.Image("projects/sat-io/open-datasets/global_consensus_landcover/mixed-other_trees");
regularly_flooded_vegetation = ee.Image("projects/sat-io/open-datasets/global_consensus_landcover/regularly_flooded_vegetation");
shrubs = ee.Image("projects/sat-io/open-datasets/global_consensus_landcover/shrubs");
consensus_LC = barren.rename('gcl_barren').addBands([cultivated_and_managed_vegetation.rename('gcl_cultivated_managed_veg')\
                                , deciduous_broadleaf_trees.rename('gcl_dec_BL'),\
                                evergreen_deciduous_needleleaf_trees.rename('gcl_dec_NL'), evergreen_broadleaf_trees.rename('gcl_EG_BL'),\
                                herbaceous_vegetation.rename('gcl_HerbV'), mixed_other_trees.rename('gcl_mixed_other')\
                                , regularly_flooded_vegetation.rename('gcl_flood_veg'),shrubs.rename('gcl_shrubs')]).clip(kenya)

# fapar
fapar = ee.ImageCollection("NOAA/CDR/AVHRR/LAI_FAPAR/V5")\
.filterDate(startDate, endDate).filterBounds(kenya).mosaic().divide(1000).select(['FAPAR','LAI'])

### Topographic related variables

In [17]:
# elevation
dsm = ee.ImageCollection("projects/sat-io/open-datasets/GLO-30").filterBounds(kenya).mosaic().clip(kenya).rename('dsm')
dtm = ee.ImageCollection("projects/sat-io/open-datasets/FABDEM").filterBounds(kenya).mosaic().clip(kenya).rename('dtm')

# Height above nearest drainage
hand = ee.ImageCollection("users/gena/global-hand/hand-100").filterBounds(kenya).mosaic().clip(kenya).rename('hand')

# Geomorphology
cti = ee.ImageCollection("projects/sat-io/open-datasets/Geomorpho90m/cti")\
.filterBounds(kenya).mosaic().clip(kenya).rename('compound_terrain_index')
tri = ee.ImageCollection("projects/sat-io/open-datasets/Geomorpho90m/tri")\
.filterBounds(kenya).mosaic().clip(kenya).rename('terrain_ruggedness_index')
slope = ee.ImageCollection("projects/sat-io/open-datasets/Geomorpho90m/slope")\
.filterBounds(kenya).mosaic().clip(kenya).rename('slope')
vrm = ee.ImageCollection("projects/sat-io/open-datasets/Geomorpho90m/vrm")\
.filterBounds(kenya).mosaic().clip(kenya).rename('vector_ruggedness_measure')
roughness = ee.ImageCollection("projects/sat-io/open-datasets/Geomorpho90m/roughness")\
.filterBounds(kenya).mosaic().clip(kenya).rename('roughness')
tpi = ee.ImageCollection("projects/sat-io/open-datasets/Geomorpho90m/tpi")\
.filterBounds(kenya).mosaic().clip(kenya).rename('topo_pos_ind')
spi = ee.ImageCollection("projects/sat-io/open-datasets/Geomorpho90m/spi")\
.filterBounds(kenya).mosaic().clip(kenya).rename('stream_power_index')

geomorph = cti.addBands([tri,slope,vrm,roughness,tpi,spi])

## Habitat, ecoregions

In [18]:
# ecoregions
ecoregions = ee.FeatureCollection("RESOLVE/ECOREGIONS/2017").filterBounds(kenya)\
.reduceToImage(ee.List(['BIOME_NUM']), ee.Reducer.first())\
.rename('ecoregion_number')

# IUCN habitat classification
# Level 1 and level 2 for the year 2015
lvl1 = ee.Image("users/Uploads/habitattypes/iucn_habitatclassification_composite_lvl1_ver004").clip(kenya)
lvl2 = ee.Image("users/Uploads/habitattypes/iucn_habitatclassification_composite_lvl2_ver004").clip(kenya)

# # Changemask for 2019
change2019_lvl1 = ee.Image("users/Uploads/habitattypes/changemasks2019/iucn_habitatclassification_2019changemask_lvl1_ver004")\
.select('comp2019').unmask(0).clip(kenya)
change2019_lvl2 = ee.Image("users/Uploads/habitattypes/changemasks2019/iucn_habitatclassification_2019changemask_lvl2_ver004")\
.select('comp2019').unmask(0).clip(kenya)

# Level 1 and level 2 for the year 2019
lvl12019 = change2019_lvl1.firstNonZero(lvl1).rename('hab2019_l1')
lvl22019 = change2019_lvl2.firstNonZero(lvl2).rename('hab2019_l2')
# eeprint(lvl22019)

In [19]:
# Bare Surface
bare_surface = ee.Image('users/geocis/BareSurfaces/BS_1980_2019').clip(kenya).regexpRename('$', '_bare_spectra')
bare_frequency = ee.Image('users/geocis/BareSurfaces/BF_1980_2019').clip(kenya).rename('bare_freq')

In [20]:
# Net Primary Productivity
NPP = ee.ImageCollection('FAO/WAPOR/2/L1_NPP_D').filterBounds(kenya).mosaic().clip(kenya).divide(1000).rename('NPP')

# Actual evapotranspiration
AET = ee.ImageCollection('FAO/WAPOR/2/L1_AETI_D').filterBounds(kenya).mosaic().clip(kenya).divide(10).rename('AET')

In [21]:
# LULC- wordcover 2020
worldcover = ee.ImageCollection("ESA/WorldCover/v100").filterBounds(kenya).mosaic().clip(kenya).rename('worldcover')

### Irrecoverable carbon

In [22]:
irrecoverable_carbon_total = ee.ImageCollection("projects/sat-io/open-datasets/irrecoverable_carbon/carbon_total")\
.filterBounds(kenya).mosaic().clip(kenya).rename('irrecoverable_C_total')
irrecoverable_carbon_soil = ee.ImageCollection("projects/sat-io/open-datasets/irrecoverable_carbon/carbon_soil")\
.filterBounds(kenya).mosaic().clip(kenya).rename('irrecoverable_C_soil')
irrecoverable_carbon_biomass = ee.ImageCollection("projects/sat-io/open-datasets/irrecoverable_carbon/carbon_biomass")\
.filterBounds(kenya).mosaic().clip(kenya).rename('irrecoverable_C_soil')
vulnerable_carbon_total = ee.ImageCollection("projects/sat-io/open-datasets/vulnerable_carbon/carbon_total")\
.filterBounds(kenya).mosaic().clip(kenya).rename('vulnerable_C_total')
vulnerable_carbon_soil = ee.ImageCollection("projects/sat-io/open-datasets/vulnerable_carbon/carbon_soil")\
.filterBounds(kenya).mosaic().clip(kenya).rename('vulnerable_C_soil')
vulnerable_carbon_biomass = ee.ImageCollection("projects/sat-io/open-datasets/vulnerable_carbon/carbon_biomass")\
.filterBounds(kenya).mosaic().clip(kenya).rename('vulnerable_C_biomass')
manageable_carbon_total = ee.ImageCollection("projects/sat-io/open-datasets/manageable_carbon/carbon_total")\
.filterBounds(kenya).mosaic().clip(kenya).rename('manageable_C_total')
manageable_carbon_soil = ee.ImageCollection("projects/sat-io/open-datasets/manageable_carbon/carbon_soil")\
.filterBounds(kenya).mosaic().clip(kenya).rename('manageable_C_soil')
manageable_carbon_biomass = ee.ImageCollection("projects/sat-io/open-datasets/manageable_carbon/carbon_biomass")\
.filterBounds(kenya).mosaic().clip(kenya).rename('manageable_C_biomass')

IC = irrecoverable_carbon_total.addBands([irrecoverable_carbon_soil, irrecoverable_carbon_biomass, vulnerable_carbon_total,\
                                         vulnerable_carbon_soil, vulnerable_carbon_biomass, manageable_carbon_total,\
                                         manageable_carbon_soil, manageable_carbon_biomass])

### Soil

In [23]:
# Soil grids
isric_bdod_mean = ee.Image("projects/soilgrids-isric/bdod_mean").clip(kenya)
isric_cec = ee.Image("projects/soilgrids-isric/cec_mean").clip(kenya)
isric_cfvo = ee.Image("projects/soilgrids-isric/cfvo_mean").clip(kenya)
isric_clay = ee.Image("projects/soilgrids-isric/clay_mean").clip(kenya)
isric_sand = ee.Image("projects/soilgrids-isric/sand_mean").clip(kenya)
isric_silt = ee.Image("projects/soilgrids-isric/silt_mean").clip(kenya)
isric_nitrogen = ee.Image("projects/soilgrids-isric/nitrogen_mean").clip(kenya)
isric_phh20 = ee.Image("projects/soilgrids-isric/phh2o_mean").clip(kenya)
isric_soc = ee.Image("projects/soilgrids-isric/soc_mean").clip(kenya)
isric_ocd = ee.Image("projects/soilgrids-isric/ocd_mean").clip(kenya)
isric_ocs = ee.Image("projects/soilgrids-isric/ocs_mean").clip(kenya)

isric = isric_bdod_mean.addBands([isric_cec, isric_cfvo, isric_clay, isric_sand, isric_silt, isric_nitrogen, isric_phh20,
                                  isric_soc, isric_ocd, isric_ocs])

In [24]:
# iSDAsoil
bedrock_depth = ee.Image("projects/sat-io/open-datasets/iSDAsoil_Africa_30m/bedrock_depth")\
.clip(kenya).regexpRename('$', '_bedrock_depth')
bulk_density = ee.Image("projects/sat-io/open-datasets/iSDAsoil_Africa_30m/bulk_density")\
.clip(kenya).regexpRename('$', '_bulk_density')
carbon_organic = ee.Image("projects/sat-io/open-datasets/iSDAsoil_Africa_30m/carbon_organic")\
.clip(kenya).regexpRename('$', '_carbon_organic')
carbon_total = ee.Image("projects/sat-io/open-datasets/iSDAsoil_Africa_30m/carbon_total")\
.clip(kenya).regexpRename('$', 'carbon_total')
cation_exchange_capacity = ee.Image("projects/sat-io/open-datasets/iSDAsoil_Africa_30m/cation_exchange_capacity")\
.clip(kenya).regexpRename('$', '_cat_ex_cap')
clay_content = ee.Image("projects/sat-io/open-datasets/iSDAsoil_Africa_30m/clay_content")\
.clip(kenya).regexpRename('$', '_clay_content')
fertility_capability_classification = ee.Image("projects/sat-io/open-datasets/iSDAsoil_Africa_30m/fcc")\
.clip(kenya).regexpRename('$', '_fertility_capability')
nitrogen_total = ee.Image("projects/sat-io/open-datasets/iSDAsoil_Africa_30m/nitrogen_total")\
.clip(kenya).regexpRename('$', '_nitrogen_total')
ph = ee.Image("projects/sat-io/open-datasets/iSDAsoil_Africa_30m/ph")\
.clip(kenya).regexpRename('$', '_ph')
sand_content = ee.Image("projects/sat-io/open-datasets/iSDAsoil_Africa_30m/sand_content")\
.clip(kenya).regexpRename('$', '_sand_content')
silt_content = ee.Image("projects/sat-io/open-datasets/iSDAsoil_Africa_30m/silt_content")\
.clip(kenya).regexpRename('$', '_silt_content')
stone_content = ee.Image("projects/sat-io/open-datasets/iSDAsoil_Africa_30m/stone_content")\
.clip(kenya).regexpRename('$', '_stone_content')
texture_class = ee.Image("projects/sat-io/open-datasets/iSDAsoil_Africa_30m/texture_class")\
.clip(kenya).regexpRename('$', '_texture_class')

isda = bedrock_depth.addBands([bulk_density, carbon_organic, carbon_total, cation_exchange_capacity, clay_content,\
                               fertility_capability_classification, nitrogen_total, ph, sand_content, silt_content,
                               stone_content, texture_class])

In [25]:
# HiHydroSoil V2
ksat = ee.ImageCollection("projects/sat-io/open-datasets/HiHydroSoilv2_0/ksat")\
.filterBounds(kenya).mosaic().clip(kenya).rename('ksat')
satfield = ee.ImageCollection("projects/sat-io/open-datasets/HiHydroSoilv2_0/sat-field")\
.filterBounds(kenya).mosaic().clip(kenya).rename('satfield')
N = ee.ImageCollection("projects/sat-io/open-datasets/HiHydroSoilv2_0/N")\
.filterBounds(kenya).mosaic().clip(kenya).rename('N')
alpha = ee.ImageCollection("projects/sat-io/open-datasets/HiHydroSoilv2_0/alpha")\
.filterBounds(kenya).mosaic().clip(kenya).rename('alpha')
crit_wilt = ee.ImageCollection("projects/sat-io/open-datasets/HiHydroSoilv2_0/crit-wilt")\
.filterBounds(kenya).mosaic().clip(kenya).rename('crit_wilt')
field_cirt = ee.ImageCollection("projects/sat-io/open-datasets/HiHydroSoilv2_0/field-crit")\
.filterBounds(kenya).mosaic().clip(kenya).rename('field_cirt')
ormc = ee.ImageCollection("projects/sat-io/open-datasets/HiHydroSoilv2_0/ormc")\
.filterBounds(kenya).mosaic().clip(kenya).rename('ormc')
stc = ee.ImageCollection("projects/sat-io/open-datasets/HiHydroSoilv2_0/stc")\
.filterBounds(kenya).mosaic().clip(kenya).rename('stc')
wcavail = ee.ImageCollection("projects/sat-io/open-datasets/HiHydroSoilv2_0/wcavail")\
.filterBounds(kenya).mosaic().clip(kenya).rename('wcavail')
wcpf2 = ee.ImageCollection("projects/sat-io/open-datasets/HiHydroSoilv2_0/wcpf2")\
.filterBounds(kenya).mosaic().clip(kenya).rename('wcpf2')
wcpf3 = ee.ImageCollection("projects/sat-io/open-datasets/HiHydroSoilv2_0/wcpf3")\
.filterBounds(kenya).mosaic().clip(kenya).rename('wcpf3')
wcpf4_2 = ee.ImageCollection("projects/sat-io/open-datasets/HiHydroSoilv2_0/wcpf4-2")\
.filterBounds(kenya).mosaic().clip(kenya).rename('wcpf4_2s')
wcres = ee.ImageCollection("projects/sat-io/open-datasets/HiHydroSoilv2_0/wcres")\
.filterBounds(kenya).mosaic().clip(kenya).rename('wcres')
wcsat = ee.ImageCollection("projects/sat-io/open-datasets/HiHydroSoilv2_0/wcsat")\
.filterBounds(kenya).mosaic().clip(kenya).rename('wcsat')

hihydro = ksat.addBands([satfield, N, alpha, crit_wilt, field_cirt, ormc, stc\
                         , wcavail, wcpf2, wcpf3, wcpf4_2, wcres, wcsat])

In [26]:
# Soil salinity
soil_salinity = ee.ImageCollection("projects/sat-io/open-datasets/global_soil_salinity").filterBounds(kenya).mosaic().clip(kenya).rename('soil_salinity')

In [27]:
# Soil moisture
soil_moisture = ee.ImageCollection('NASA_USDA/HSL/SMAP10KM_soil_moisture')\
.filterDate(startDate,endDate).filterBounds(kenya)\
.select(['ssm', 'susm']).median().clip(kenya)

In [28]:
# night time lights
viirs_ntl = ee.ImageCollection("projects/sat-io/open-datasets/npp-viirs-ntl").filterBounds(kenya).mosaic().clip(kenya).rename('viirs_ntl')

### Socio-economic variables

In [29]:
# social data - rwi, gdp, hdi
rwi = ee.FeatureCollection("projects/sat-io/open-datasets/facebook/relative_wealth_index")\
.filterBounds(kenya)\
.reduceToImage(['rwi'], ee.Reducer.first())\
.unmask()\
.rename('rwi')

gdp_ppp = ee.Image("projects/sat-io/open-datasets/GRIDDED_HDI_GDP/GDP_PPP_1990_2015_5arcmin_v2")\
.clip(kenya).select('b26').rename('GDP')

hdi = ee.Image("projects/sat-io/open-datasets/GRIDDED_HDI_GDP/HDI_1990_2015_v2")\
.clip(kenya).select('b26').rename('HDI')

global_CISI = ee.Image("projects/sat-io/open-datasets/CISI/global_CISI")\
.clip(kenya).rename('CISI')

infrastructure = ee.ImageCollection("projects/sat-io/open-datasets/CISI/amount_infrastructure")\
.filterBounds(kenya).mosaic().clip(kenya).rename('amt_infra')

#### Habitat Heterogeneity

In [30]:
cov = ee.Image("projects/sat-io/open-datasets/global_habitat_heterogeneity/coefficient_of_variation_1km")\
.clip(kenya).rename('cov')
contrast = ee.Image("projects/sat-io/open-datasets/global_habitat_heterogeneity/contrast_1km")\
.clip(kenya).rename('contrast')
corr = ee.Image("projects/sat-io/open-datasets/global_habitat_heterogeneity/correlation_1km")\
.clip(kenya).rename('corr')
dissimilarity = ee.Image("projects/sat-io/open-datasets/global_habitat_heterogeneity/dissimilarity_1km")\
.clip(kenya).rename('dissimilarity')
entropy = ee.Image("projects/sat-io/open-datasets/global_habitat_heterogeneity/entropy_1km")\
.clip(kenya).rename('entropy')
homogeneity = ee.Image("projects/sat-io/open-datasets/global_habitat_heterogeneity/homogeneity_1km")\
.clip(kenya).rename('homogeneity')
maximum = ee.Image("projects/sat-io/open-datasets/global_habitat_heterogeneity/maximum_1km")\
.clip(kenya).rename('maximum')
mean = ee.Image("projects/sat-io/open-datasets/global_habitat_heterogeneity/mean_1km")\
.clip(kenya).rename('mean')
pielou = ee.Image("projects/sat-io/open-datasets/global_habitat_heterogeneity/pielou_1km")\
.clip(kenya).rename('pielou')
hrange = ee.Image("projects/sat-io/open-datasets/global_habitat_heterogeneity/range_1km")\
.clip(kenya).rename('range')
shannon = ee.Image("projects/sat-io/open-datasets/global_habitat_heterogeneity/shannon_1km")\
.clip(kenya).rename('shannon')
simpson = ee.Image("projects/sat-io/open-datasets/global_habitat_heterogeneity/simpson_1km")\
.clip(kenya).rename('simpson')
sd = ee.Image("projects/sat-io/open-datasets/global_habitat_heterogeneity/standard_deviation_1km")\
.clip(kenya).rename('sd')
uniformity = ee.Image("projects/sat-io/open-datasets/global_habitat_heterogeneity/uniformity_1km")\
.clip(kenya).rename('uniformity')
variance = ee.Image("projects/sat-io/open-datasets/global_habitat_heterogeneity/variance_1km")\
.clip(kenya).rename('variance')

heterogeneity = cov.addBands([contrast, corr, dissimilarity, entropy, homogeneity, maximum,\
                              mean, pielou, hrange, shannon, simpson,sd, uniformity, variance])

#### Cloud frequency

In [31]:
# EarthEnv Cloud Frequency v1.0
cloud_forest_prediction = ee.Image("projects/sat-io/open-datasets/gcc/MODCF_CloudForestPrediction")\
.clip(kenya).rename('cloud_forest_prediction')
interannual_sd = ee.Image("projects/sat-io/open-datasets/gcc/MODCF_interannualSD")\
.clip(kenya).rename('interannual_sd')
intrannual_sd = ee.Image("projects/sat-io/open-datasets/gcc/MODCF_intraannualSD")\
.clip(kenya).rename('intrannual_sd')
mean_annual = ee.Image("projects/sat-io/open-datasets/gcc/MODCF_meanannual")\
.clip(kenya).rename('mean_annual')
monthly_mean = ee.ImageCollection("projects/sat-io/open-datasets/gcc/MODCF_monthlymean")\
.filterBounds(kenya).mosaic().clip(kenya).rename('monthly_mean')
seasonality_concentration = ee.Image("projects/sat-io/open-datasets/gcc/MODCF_seasonality_concentration")\
.clip(kenya).rename('seasonality_concentration')
seasonality_theta = ee.Image("projects/sat-io/open-datasets/gcc/MODCF_seasonality_theta")\
.clip(kenya).rename('seasonality_theta')
seasonality_visct = ee.Image("projects/sat-io/open-datasets/gcc/MODCF_seasonality_visct")\
.clip(kenya).rename('seasonality_visct')
spatial_sd_1deg = ee.Image("projects/sat-io/open-datasets/gcc/MODCF_spatialSD_1deg")\
.clip(kenya).rename('spatial_sd_1deg')

cloud = cloud_forest_prediction.addBands([interannual_sd, intrannual_sd, mean_annual, monthly_mean,\
                                          seasonality_concentration, seasonality_theta,\
                                          seasonality_visct, spatial_sd_1deg])

#### Evapotranspiration

In [32]:
et_monthly = ee.ImageCollection("projects/sat-io/open-datasets/global_et0/global_et0_monthly")\
.filterBounds(kenya).mosaic().clip(kenya).rename('et_monthly')
et_yearly = ee.Image("projects/sat-io/open-datasets/global_et0/global_et0_yearly")\
.clip(kenya).rename('et_yearly')
et_yearly_sd = ee.Image("projects/sat-io/open-datasets/global_et0/global_et0_yearly_sd")\
.clip(kenya).rename('et_yearly_sd')
et = et_monthly.addBands([et_yearly, et_yearly_sd])

#### Aridity Index

In [33]:
aridity_index_yearly = ee.Image("projects/sat-io/open-datasets/global_ai/global_ai_yearly")\
.clip(kenya).rename('aridity_index_yearly')
aridity_index_monthly = ee.ImageCollection("projects/sat-io/open-datasets/global_ai/global_ai_monthly")\
.filterBounds(kenya).mosaic().clip(kenya).rename('aridity_index_monthly')
aridity = aridity_index_yearly.addBands(aridity_index_monthly)
# eeprint(aridity)

#### Burn probability

In [34]:
burn_probability = ee.Image("users/keikonomura/fire/LBA_CU_2019_20200415_C01_V01_BP_L8").clip(kenya).rename('burn_probability')

#### Cattle grazing

In [35]:
cattle_grazing = ee.Image('projects/ee-geethensingh/assets/GrazingDensity/5_Ct_2010_Da').rename('cattle_grazing')

#### Exclosures

In [36]:
exclosures = ee.FeatureCollection('users/geethensingh/NS/Exclosures').map(lambda ft: ft.set('presence', 1))\
                .reduceToImage(['presence'], ee.Reducer.first()).rename('exclosures').unmask(0)

#### Compile covariates into a single image

In [37]:
# All covariates
coords = ee.Image.pixelCoordinates('EPSG:4326')
covariates = tree_canopy.addBands([burn_probability, aridity, et, cloud,heterogeneity, soil_moisture, rwi, gdp_ppp, hdi, global_CISI,\
                                   infrastructure, soil_salinity, viirs_ntl, hihydro,isda, isric,\
                                   IC, bare_surface, bare_frequency, NPP, AET,worldcover, ecoregions,\
                                   lvl12019, lvl22019, VH, VV, dsm, dtm,hand, geomorph, canopy_height,\
                                   ch_standard_deviation, tree_type, consensus_LC,fapar, S2, L8, L9, S2VI,\
                                   Palsar, cattle_grazing, exclosures, coords])

# eeprint(covariates.bandNames())

In [38]:
# print the number of bands
eeprint(covariates.bandNames().size())

#### Variables by spatial resolution (m).  

- <10 - monthlyPlanet, exclosures
- 10 - S2, S2VI, VH, VV, tree_canopy, canopy_height, ch_standard_deviation, worldcover
- 25 - Palsar
- 30 - L8, L9, dsm, dtm, hand, bare_surface, bare_frequency, isda
- 90 - geomorph
- 250 - NPP, AET, isric, hihydro
- 300 - IC
- 500 - tree_type, viirs_ntl
- 1000 - consensus_LC, soil_salinity, heterogeneity, cloud, et, aridity
- 2400 - rwi
- 5000 - fapar, ecoregions, lvl12019, lvl22019
- 11000 - global_CISI, gdp_ppp, hdi, infrastructure, soil_moisture, cattle_grazing

In [38]:
# Covariates with a spatial resolution less than 30 metres
ftslt30m = VH.addBands([VV, dsm, dtm, hand, exclosures, S2, L8, L9, S2VI, canopy_height,\
ch_standard_deviation, worldcover, bare_surface, bare_frequency,isda])

In [41]:
eeprint(ftslt30m.bandNames())

## Extract covariataes

In [39]:
projection = target.projection()
# Extract data
fc = target.sample(**{'dropNulls': True, 'factor': None,\
                    'numPixels': None,  'region': kenya,\
                'scale': 1000,'projection': projection,'geometries':True})
spcvGrid, _ = createGrid(50000, aoi = kenya, vect= False)
spcvGrid = spcvGrid.rename('id')

In [41]:
filename = r'C:\Users\coach\myfiles\conservation\Handover\AGBD\Data\inputs\biomass.parquet'
covs = target.addBands([spcvGrid, covariates])
properties = covs.bandNames().getInfo()

nvars = len(list(properties))
if os.path.exists(filename):
    df = pd.read_parquet(filename)
    resume_from = df.shape[1]+1
else:
    resume_from = 0
    df = pd.DataFrame()

print('Column Count:', resume_from)
    
for covariate in tqdm(range(resume_from,nvars)):
    covariate = str(properties[covariate])
    var = covs.select(covariate)
    # scale = var.projection().nominalScale()
    print(covariate)
    data = var.reduceRegions(fc, ee.Reducer.first(), 1000, tileScale = 4)
    result = pd.DataFrame(data.aggregate_array('first').getInfo(), columns = [covariate])
    df = pd.concat([df, result], axis=1)
    print(df.shape)
    df.to_parquet(filename)
print(df.shape)

Column Count: 243


  0%|          | 0/2 [00:00<?, ?it/s]

x
(411520, 243)
y
(411520, 244)
(411520, 244)


In [39]:
def extract_eemont_indices(indices, imageCollection, platform, append, filename, batch = 5):
    """
    Compute and extract optical indices based on the eemont package.  
    
    Args:
        indices (string): Supported strings 'all', 'vegetation'
        imageCollection (ee.ImageCollection): the imagecollection to compute the spectral index on.
        platform (string): one of 'Sentinel-2', 'Landsat-OLI', 'Landsat-TM', 'Landsat-ETM+', 'MODIS', 'Planet-Fusion'
        append (string): append to output band name
        batch (integer): The number of indices to compute and extract at any one time
    
    Returns:
        A parquet file
        
    """
    if os.path.exists(filename):
        df = pd.read_parquet(filename)
        resume_from = df.shape[1]+1
    else:
        resume_from = 0
        df = pd.DataFrame()
    if indices == 'all':
        filteredindices = list()
        platform = platform
        for index in eemont.listIndices():
            indices = eemont.indices()
            if platform in list(indices[index].platforms):
                filteredindices.append(index)
        filteredindices.remove('NIRv')
        filteredindices.remove('NIRvP')
        nindices = len(filteredindices)
        for subset in tqdm(range(resume_from, nindices, batch)):
            if subset+batch>nindices:
                indicesSubset = filteredindices[subset:nindices]
            else:
                indicesSubset = filteredindices[subset:subset+batch]
                
            indicesSubset = [sub + append for sub in indicesSubset]
            finalIndices = list(set(indicesSubset)-set(properties))
            finalIndices = [i.removesuffix(append) for i in finalIndices]
            if len(finalIndices)>0:
                print(f'Extracting: Variable {subset}-{finalIndices}')
                output = imageCollection.spectralIndices(finalIndices, drop=True).median().regexpRename('$', append)
                data = output.reduceRegions(fc, ee.Reducer.first(), 1000, tileScale = 4)
                result = pd.DataFrame(data.aggregate_array('first').getInfo(), columns = finalIndices)
                df = pd.concat([df, result], axis=1)
                print(df.shape)
                df.to_parquet(filename)

In [51]:
fn2 = r'C:\Users\coach\myfiles\conservation\Handover\AGBD\Data\scratch\biomass_indicess2.parquet'  
covs = target.addBands(covariates)
properties = covs.bandNames().getInfo()
extract_eemont_indices(indices= 'all', imageCollection = s2, platform = 'Sentinel-2', append = '_s2', filename = fn2, batch = 1)

  0%|                                                                                           | 0/25 [00:00<?, ?it/s]

Extracting: Variable 144-['TGI']
(411520, 144)


  4%|███▎                                                                               | 1/25 [00:20<08:16, 20.69s/it]

Extracting: Variable 145-['TRRVI']
(411520, 145)


  8%|██████▌                                                                           | 2/25 [03:16<42:58, 112.09s/it]

...

 92%|██████████████████████████████████████████████████████████████████████████▌      | 23/25 [55:38<04:55, 147.62s/it]

Extracting: Variable 167-['mND705']
(411520, 167)


 96%|█████████████████████████████████████████████████████████████████████████████▊   | 24/25 [58:22<02:32, 152.40s/it]

Extracting: Variable 168-['mSR705']
(411520, 168)


100%|███████████████████████████████████████████████████████████████████████████████| 25/25 [1:00:51<00:00, 146.04s/it]


In [52]:
fn3 = r'C:\Users\coach\myfiles\conservation\Handover\AGBD\Data\scratch\biomass_indicesl8.parquet'  
covs = target.addBands(covariates)
properties = covs.bandNames().getInfo()
extract_eemont_indices(indices= 'all', imageCollection = l8, platform = 'Landsat-OLI', append = '_l9', filename = fn3, batch = 1)

  0%|                                                                                          | 0/148 [00:00<?, ?it/s]

Extracting: Variable 1-['AFRI2100']


  1%|▌                                                                               | 1/148 [01:14<3:02:56, 74.67s/it]

(411520, 1)
Extracting: Variable 2-['ARVI']


  1%|█                                                                               | 2/148 [02:19<2:47:11, 68.71s/it]

...

 99%|████████████████████████████████████████████████████████████████████████████▉ | 146/148 [2:46:31<02:15, 67.51s/it]

Extracting: Variable 147-['kRVI']
(411520, 147)


 99%|█████████████████████████████████████████████████████████████████████████████▍| 147/148 [2:47:41<01:08, 68.22s/it]

Extracting: Variable 148-['kVARI']
(411520, 148)


100%|██████████████████████████████████████████████████████████████████████████████| 148/148 [2:48:48<00:00, 68.43s/it]


In [53]:
fn4 = r'C:\Users\coach\myfiles\conservation\Handover\AGBD\Data\scratch\biomass_indicesl9.parquet'  
covs = target.addBands(covariates)
properties = covs.bandNames().getInfo()
extract_eemont_indices(indices= 'all', imageCollection = l9, platform = 'Landsat-OLI', append = '_l9a', filename = fn4, batch = 1)

  0%|                                                                                          | 0/148 [00:00<?, ?it/s]

Extracting: Variable 1-['AFRI2100']


  1%|▌                                                                               | 1/148 [00:50<2:04:37, 50.87s/it]

(411520, 1)
Extracting: Variable 2-['ARVI']


  1%|█                                                                               | 2/148 [01:42<2:04:43, 51.26s/it]

...

 99%|████████████████████████████████████████████████████████████████████████████▉ | 146/148 [2:03:07<01:40, 50.46s/it]

Extracting: Variable 147-['kRVI']
(411520, 147)


 99%|█████████████████████████████████████████████████████████████████████████████▍| 147/148 [2:03:57<00:50, 50.31s/it]

Extracting: Variable 148-['kVARI']
(411520, 148)


100%|██████████████████████████████████████████████████████████████████████████████| 148/148 [2:04:47<00:00, 50.59s/it]


### Read in extracted data, merge into a single dataframe and save as feather

In [42]:
import pandas as pd

df1 = pd.read_parquet(r'C:\Users\coach\myfiles\conservation\Handover\AGBD\Data\scratch\biomass.parquet')
print(df1.shape)
df2 = pd.read_parquet(r'C:\Users\coach\myfiles\conservation\Handover\AGBD\Data\scratch\biomass_indicess2.parquet').add_suffix('_s2')
print(df2.shape)
df3 = pd.read_parquet(r'C:\Users\coach\myfiles\conservation\Handover\AGBD\Data\scratch\biomass_indicesl8.parquet').add_suffix('_l8')
print(df3.shape)
df4 = pd.read_parquet(r'C:\Users\coach\myfiles\conservation\Handover\AGBD\Data\scratch\biomass_indicesl9.parquet').add_suffix('_l9')
print(df4.shape)

df = pd.concat([df1, df2, df3, df4], axis = 1)
print(df.shape)

(411520, 244)
(411520, 168)
(411520, 148)
(411520, 148)
(411520, 708)


In [44]:
df.to_feather(r'C:\Users\coach\myfiles\conservation\Handover\AGBD\Data\inputs\biomass_708.feather')

Wishlist

Variables to add:
- Texture metrics
- Radar derived indices
- VODCA
- Phenology
- Deforestation occurence

## 5. Extracting Image Data for inference

In [5]:
from pathlib import Path
import ee
from tqdm.auto import tqdm
import geedim as gd
from geedim.download import BaseImage

ee.Initialize(opt_url="https://earthengine-highvolume.googleapis.com")

In [36]:
# Top 82 features (use for model with artefacts/ model with covariates > 30m resolution)

features = ['hand',
 'normalized_VH',
 'aridity_index_yearly',
 'gcl_dec_BL',
 'GDP',
 'gcl_EG_BL',
 'mean_annual',
 'NDYI_s2',
 'b1_cat_ex_cap',
 'aridity_index_monthly',
 'WDVI_l8',
 'B8_s2',
 'gcl_mixed_other',
 'NBR2_l8',
 'EMBI_l8',
 'ssm',
 'LC_Type1',
 'NDWI_l8',
 'vector_ruggedness_measure',
 'S2REP_s2',
 'seasonality_visct',
 'NDDI_l8',
 'intrannual_sd',
 'soc_5-15cm_mean',
 'phh2o_0-5cm_mean',
 'swir1_bare_spectra',
 'CISI',
 'NMDI_l9',
 'cfvo_60-100cm_mean',
 'ST_B10_l9',
 'NDYI_l9',
 'OSAVI_l8',
 'NIRvH2_s2',
 'gcl_HerbV',
 'ormc',
 'clay_15-30cm_mean',
 'NDBaI_l8',
 'susm',
 'cloud_forest_prediction',
 'SAVI2_s2',
 'interannual_sd',
 'B5_s2',
 'MLSWI27_l8',
 'MLSWI26_l9',
 'kVARI_l9',
 'b1_texture_class',
 'VIG_l8',
 'b2_ph',
 'hab2019_l1',
 'VIG_s2',
 'MCARIOSAVI_s2',
 'MNDWI_l8',
 'soc_60-100cm_mean',
 'UI_l9',
 'NormG_l8',
 'TCARIOSAVI705_s2',
 'b1_fertility_capability',
 'cfvo_5-15cm_mean',
 'vulnerable_C_total',
 'silt_30-60cm_mean',
 'NDISIg_l9',
 'bare_freq',
 'AET',
 'FCVI_s2',
 'NDDI_l9',
 'NDSaII_l9',
 'CVI_l9',
 'MCARIOSAVI705_s2',
 'b2_silt_content',
 'alpha',
 'NDSaII_s2',
 'EVI_s2',
 'cec_15-30cm_mean',
 'NBRT2_l9',
 'GNDVI_l9',
 'NBRT1_l8',
 'wcpf2',
 'kEVI_l8',
 'nitrogen_0-5cm_mean',
 'corr',
 'bdod_15-30cm_mean',
 'worldcover']

In [41]:
# Top 61 high res features
features = ['NDMI_s2',
 'MRBVI_l9',
 'NDYI_l9',
 'worldcover',
 'ND705_s2',
 'hand',
 'kEVI_s2',
 'b2_carbon_organic',
 'b2_texture_class',
 'MNDWI_l9',
 'bare_freq',
 'MLSWI27_l8',
 'b1_silt_content',
 'b1_cat_ex_cap',
 'PISI_s2',
 'NDVIT_l8',
 'kEVI_l9',
 'MLSWI27_s2',
 'b1_texture_class',
 'NDBaI_l8',
 'kEVI_l8',
 'BCC_s2',
 'normalized_VV',
 'NDWIns_l9',
 'NMDI_l9',
 'NDDI_l9',
 'b3_ph',
 'TGI_l9',
 'NDVIMNDWI_l8',
 'AVI_l8',
 'b1_fertility_capability',
 'NGRDI_l8',
 'NIRvH2_l8',
 'NDDI_l8',
 'TCARIOSAVI_s2',
 'MLSWI26_l8',
 'kVARI_s2',
 'MLSWI26_s2',
 'TCARIOSAVI705_s2',
 'SR_B2_l9',
 'AVI_s2',
 'SR_B5_l8',
 'SR_l8',
 'b3_sand_content',
 'kIPVI_s2',
 'DBI_l8',
 'NDSII_s2',
 'NBRT1_l9',
 'b3_carbon_organic',
 'BAI_s2',
 'NWI_l9',
 'b3_bulk_density',
 'SR_B4_l8',
 'nir_bare_spectra',
 'SIPI_s2',
 'b2_stone_content',
 'NBLI_l9',
 'MCARIOSAVI705_s2',
 'VIG_s2',
 'NDDI_s2',
 'NSDS_s2']

In [42]:
aoi = aoi = ee.FeatureCollection("users/geethensingh/NS/Boundary").geometry().buffer(10000).bounds()

with_s2 = [x.removesuffix('_s2') for x in features if x.endswith('_s2')]
S2VI = s2.filterBounds(aoi).spectralIndices(with_s2, drop=True).median().regexpRename('$', '_s2')

with_l8 = [x.removesuffix('_l8') for x in features if x.endswith('_l8')]
L8VI = l8.filterBounds(aoi).spectralIndices(with_l8, drop=True).median().regexpRename('$', '_l8')

with_l9 = [x.removesuffix('_l9') for x in features if x.endswith('_l9')]
L9VI = l9.filterBounds(aoi).spectralIndices(with_l9, drop=True).median().regexpRename('$', '_l9')

In [43]:
covs = covariates.addBands([S2VI, L8VI, L9VI]).select(features)
eeprint(covs.bandNames().size())

In [44]:
# download from gee directly to local drive
# If this fails, export to asset first

aoi = ee.FeatureCollection("users/geethensingh/NS/Boundary").geometry().buffer(10000).bounds()
filename = Path('C:/Users/coach/myfiles/conservation/data/biomass_results/added_covs/conservancy2_10m_61fts.tif')
BaseImage(covs).download(filename, crs='EPSG:4326', region= aoi, scale=10, overwrite=True, num_threads=20, dtype= 'float64')

Consider adjusting `region`, `scale` and/or `dtype` to reduce the conservancy2_10m_61fts.tif download size (raw: 16.50 GB).


conservancy2_10m_61fts.tif: |                                         | 0.00/16.5G (raw) [  0.0%] in 00:00 (et…

OSError: Error downloading tile: User memory limit exceeded.
The `max_tile_size` or `max_tile_dim` parameters can be decreased to work around this error.

In [48]:
#Alternatively export image to assets first then export to drive/download via geedim
task = ee.batch.Export.image.toAsset(**{'image':covs, 'description': 'conservancy_61', 'assetId':'users/geethensingh/conservancy_10m_61v', 'region': aoi, 'scale': 10, 'maxPixels': 1e13})
task.start()

In [50]:
task.status()

{'state': 'COMPLETED',
 'description': 'conservancy_61',
 'creation_timestamp_ms': 1668059736384,
 'update_timestamp_ms': 1668062132534,
 'start_timestamp_ms': 1668059760642,
 'task_type': 'EXPORT_IMAGE',
 'destination_uris': ['https://code.earthengine.google.com/?asset=projects/earthengine-legacy/assets/users/geethensingh/conservancy_10m_61v'],
 'attempt': 1,
 'batch_eecu_usage_seconds': 433049,
 'id': 'KNVSRUK4MCDSD736JC3WNVPA',
 'name': 'projects/earthengine-legacy/operations/KNVSRUK4MCDSD736JC3WNVPA'}

In [6]:
img = ee.Image('users/geethensingh/conservancy_10m_61v')
aoi = ee.FeatureCollection("users/geethensingh/NS/Boundary").geometry().buffer(10000).bounds()
filename = Path('C:/Users/coach/myfiles/conservation/data/biomass_results/added_covs/conservancy2_10m_61fts.tif')
BaseImage(img).download(filename, crs='EPSG:4326', region= aoi, scale=10, overwrite=True, num_threads=20)

Consider adjusting `region`, `scale` and/or `dtype` to reduce the conservancy2_10m_61fts.tif download size (raw: 16.50 GB).


conservancy2_10m_61fts.tif: |                                         | 0.00/16.5G (raw) [  0.0%] in 00:00 (et…

There is no STAC entry for: users/geethensingh/conservancy_10m_61v


## 6. Extract variables for field site exploration

In [5]:
from geedim.download import BaseImage
from pathlib import Path
from tqdm import tqdm
import ee
import geemap

aoi = ee.FeatureCollection("users/geethensingh/NS/Boundary").geometry().buffer(10000).bounds()
Map = geemap.Map()
Map.addLayer(aoi)
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [8]:
# worldcover
lc = ee.ImageCollection("ESA/WorldCover/v100").filterBounds(aoi).mosaic().rename('worldcover')

# brandt

tree_canopy = ee.ImageCollection('projects/wri-datalab/TML').filterBounds(aoi).mosaic().rename('tree_canopy')
# rainfall
years = list(range(2016,2021))

precipitation = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY").filterBounds(aoi)  
precipitation = ee.Image(ee.ImageCollection(ee.List(years).map(lambda year: precipitation\
.filterDate(ee.Number(year).format(), ee.Number(year).add(1).format()).sum())).mean().rename('mm_precipitation')) 

# height
dsm = ee.ImageCollection("projects/sat-io/open-datasets/GLO-30").filterBounds(aoi).mosaic().rename('dsm')

data = lc.addBands([tree_canopy, precipitation, dsm])
filename = Path('C:/Users/coach/myfiles/conservation/data/Conservancy_4covs_Nicky.tif')
BaseImage(data).download(filename, crs='EPSG:4326', region= aoi, scale=10, overwrite=True, num_threads=20, dtype= 'float64')

Consider adjusting `region`, `scale` and/or `dtype` to reduce the Conservancy_4covs_Nicky.tif download size (raw: 1.08 GB).


Conservancy_4covs_Nicky.tif: |                                        | 0.00/1.08G (raw) [  0.0%] in 00:00 (et…

There is no STAC entry for: None


## 7. Extract variables for SOC field site exploration

In [4]:
from geedim.download import BaseImage
from pathlib import Path
from tqdm import tqdm
import ee
import geemap, eerepr
ee.Authenticate()
ee.Initialize()


aoi = ee.FeatureCollection("users/geethensingh/NS/Boundary").geometry().buffer(10000).bounds()
Map = geemap.Map()
Map.addLayer(aoi)
Map

Enter verification code:  4/1AfgeXvvP045z3j6AlyIb42yMbjDsvhcmuofeCiCR4bGXmllfdkjd_9PSSyM



Successfully saved authorization token.


Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [11]:
# worldcover
lc = ee.ImageCollection("ESA/WorldCover/v100").filterBounds(aoi).mosaic().rename('worldcover')

# tree canopy height (lang)
canopy_height = ee.Image("users/nlang/ETH_GlobalCanopyHeight_2020_10m_v1").rename('lang_CHM')

# rainfall
years = list(range(2016,2021))
precipitation = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY").filterBounds(aoi)  
precipitation = ee.Image(ee.ImageCollection(ee.List(years).map(lambda year: precipitation\
.filterDate(ee.Number(year).format(), ee.Number(year).add(1).format()).sum())).mean().rename('mm_precipitation')) 

# height
dtm = ee.ImageCollection("projects/sat-io/open-datasets/FABDEM").filterBounds(aoi).mosaic().rename('dtm')

# Net Primary Productivity
NPP = ee.ImageCollection('FAO/WAPOR/2/L1_NPP_D').filterBounds(aoi).mosaic().divide(1000).rename('NPP')

# bare surface frequency
bare_frequency = ee.Image('users/geocis/BareSurfaces/BF_1980_2019').rename('bare_freq')

# Catttle grazing density
cattle_grazing = ee.Image('projects/ee-geethensingh/assets/GrazingDensity/5_Ct_2010_Da').rename('cattle_grazing')

# Topographic diversity
TD = ee.Image("CSP/ERGo/1_0/Global/SRTM_topoDiversity").rename('topo_diversity')

# MODIS NIght temperature
startDate = '2016-01-01'
endDate = '2021-01-01'
temp = ee.ImageCollection("MODIS/061/MOD11A1").filterBounds(aoi).filterDate(startDate, endDate)\
.select('LST_Night_1km').mean().rename('night_temp')

# NDVI
ndvi = ee.ImageCollection("MODIS/061/MOD13Q1").filterBounds(aoi).filterDate(startDate, endDate)\
.select('NDVI').mean().rename('ndvi')

# sand content
sand_content = ee.Image("projects/sat-io/open-datasets/iSDAsoil_Africa_30m/sand_content")\
.regexpRename('$', '_sand_content')

data = lc.addBands([canopy_height, precipitation, dtm, NPP, bare_frequency, cattle_grazing, TD, temp, ndvi, sand_content])
data.bandNames()

filename = Path('C:/Users/coach/myfiles/conservation/data/Conservancy_SOCcovs.tif')
BaseImage(data).download(filename, crs='EPSG:4326', region= aoi, scale=10, overwrite=True, num_threads=20, dtype= 'float64')

Consider adjusting `region`, `scale` and/or `dtype` to reduce the Conservancy_SOCcovs.tif download size (raw: 3.79 GB).


Conservancy_SOCcovs.tif: |                                            | 0.00/3.79G (raw) [  0.0%] in 00:00 (et…

There is no STAC entry for: None
